In [2]:
# importamos librerías a utlizar
import numpy as np
import pymongo
import pandas as pd
from pandas.io.json import json_normalize

In [4]:
# conectamos a la bd
from pymongo import MongoClient
from urllib.parse import quote_plus

mongo_user = "dev"
mongo_password = "dev12345"
mongo_url = "ds123454.mlab.com:23454/macri-discursos"
monto_url_stats = "ds115283.mlab.com:15283/discursos-stats"
host = "mongodb://ds123454.mlab.com"
port = 23454
db_name = "macri-discursos"

uri =  "mongodb://{}:{}@{}".format(quote_plus(mongo_user), quote_plus(mongo_password), mongo_url)

client = MongoClient(uri)
db = client[db_name]

In [6]:
# colecciones que vamos a utilizar
collections = ["speeches", "exchanges", "speeches_stats"]

In [58]:
# obtenemos todas las cotizaciones del dolar
exchanges = list(db.exchanges.find({}))

In [59]:
# preprocesamos la info
exchanges = pd.DataFrame(exchanges).drop(["_id", "__v"], axis=1).set_index('date')
exchanges.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1128 entries, 2015-12-01 03:00:00 to 2018-12-01 03:00:00
Data columns (total 1 columns):
price    1128 non-null float64
dtypes: float64(1)
memory usage: 17.6 KB


In [108]:
# obtenemos las variaciones del dolar agrupadas cada 2 días, gracias a pandas y time-series
variacion = exchanges.resample("2D").apply(lambda df: df.iloc[-1].price - df.iloc[0].price)

In [109]:
# nos quedamos con las mayores variaciones (positivas y negativas)
variacion_negativa = variacion[(variacion < 0)].sort_values().head(10)
variacion_positiva = variacion[(variacion>0)].sort_values(ascending=False).head(10)

In [116]:
print(variacion_negativa, variacion_positiva)

date
2018-09-30   -1.813256
2018-09-20   -1.030452
2018-10-30   -0.830880
2018-10-04   -0.808928
2018-10-10   -0.578996
2017-12-28   -0.565945
2018-09-18   -0.452956
2018-05-03   -0.425976
2018-09-16   -0.423495
2016-03-10   -0.387052
dtype: float64 date
2018-08-29    4.731094
2018-05-13    1.948073
2018-11-25    1.496793
2018-09-26    1.316010
2018-09-02    1.164031
2018-09-12    1.144484
2018-08-09    1.131538
2018-09-24    0.989994
2018-06-28    0.911033
2018-07-08    0.838979
dtype: float64


In [107]:
# forma de obtener los precios de las cotizaciones indexadas por fechas.
exchanges.loc['2018-08-28':'2018-08-29']

,price
date,
2018-08-28 03:00:00,31.470486
2018-08-29 03:00:00,34.001969
